In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Load model_path from config.py
from config import model_path

# Choose llama3 model path
llama3_path = model_path["llama3.1"]

# Load tokenizer and model using the "unsloth" approach if needed
# If you want to use Unsloth's AutoModelForCausalLM, make sure the pip package is installed and imported.
# Here is a generic loading example (you can substitute with Unsloth's classes if needed):

tokenizer = AutoTokenizer.from_pretrained(llama3_path)
model = AutoModelForCausalLM.from_pretrained(llama3_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [7]:
from env_adaptors.webshop_llama_atopter import WebshopLlamaAtopter
a = WebshopLlamaAtopter("A")
a.initialize_env()

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 88971.70it/s]
/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


Loaded 6910 goals.
